(interval_dependency)=
# Interval dependency

In [1]:
# remove this after finished
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext rich
from pyuncertainnumber import pba

**Interval dependency** is a known caveat of [interval arithmetic](https://en.wikipedia.org/wiki/Interval_arithmetic). Despite the many appealing features of interval analysis, the presence of the dependency issue requires analysts to be cautious with their calculations since it may lead to unexpected inflation of uncertainty in the response. We will be demonstrating the *dependency issue* in this notebook.

## Interval dependency

It should be noted that $X^2$ is not the same as  $X \cdot X$ under interval arithmetic, though they are identical under ordinary arithmetic. We can have a quick check on the results of these two functions respectively

In [3]:
X = pba.I(-1,1)

In [4]:
X**2

[0.0,1.0]

In [5]:
X * X

[-1.0,1.0]

```{note}
Apparently $X \cdot X$ leads to an overestimation, as explained in Moore et al. (2009) as "Namely, if we assume x is an unknown number known to lie in the interval $X$, then, when we form the product $ x \cdot x$, the $x$ in the second factor, although known only to lie in $X$ must be the same as the $x$ in the first factor, whereas, in the definition of the interval product $X \cdot X$, it is assumed that the values in the first factor and the values in the second factor vary independently"
```

## Repeated variable problem

Generally, analysts shall be very careful when the response function involves repeated variables, since interval arithmeic will likely to yield inflated results. Below shows another example function $x(1-x)$.


<figure style="text-align: center;">
    <img src="../../_static/function_hint.png" width="1000">
</figure>

Mathematically, under ordinary arithmetic, we can write in a few equivalent manners:

```{math}
\begin{align}
f(x) &= x(1-x) \newline
g(x) &= x - x^2 \newline
h(x) &= \frac{1}{4} - (x - \frac{1}{2})^2
\end{align}
```

However, as demonstrated below, the interval-valued extensions of these functions will give different results:
<!-- $$
f(x) = x(1-x)
$$

$$
g(x) = x - x^2
$$

$$
h(x) = \frac{1}{4} - (x - \frac{1}{2})^2
$$ -->

In [6]:
X = pba.I(0,1)
def f(x): return x * (1 - x)
def g(x): return x - x**2
def h(x): return 1/4 - (x - 1/2)**2

In [7]:
f(X)

[0.0,1.0]

In [8]:
g(X)

[-1.0,1.0]

In [9]:
h(X)

[0.0,0.25]

It turns out that the united extension can be yielded from $h(x) = \frac{1}{4} - (x - \frac{1}{2})^2$. One can easily tell the correct result from the figure above.

## Common strategies against repeated variable problem

Commonly, one can try rewriting the considered function, if known, to remove the dependency as shown above, or to resort to other interval approaches, such as *subinterval* or *optimisation-based* approach.

```{tip}
`pyuncertainnumber` provides a bespoke function [b2b](https://pyuncertainnumber.readthedocs.io/en/latest/autoapi/pyuncertainnumber/index.html#pyuncertainnumber.b2b) for a range of interval propagation strategies. 
```

In [10]:
import pyuncertainnumber as pun

In [ ]:
y_sub = pun.b2b(vars=X, 
        func=f,
        interval_strategy="subinterval",
        subinterval_style="direct",
        n_sub=5000
       )

In [12]:
y_sub

[0.0,0.2501]

In [13]:
%%capture
y_opt = pun.b2b(vars=X, 
        func=f,
        interval_strategy="bo",
       )

In [14]:
y_opt

[0.0,0.2499999760755748]

```{note}
As shown, for *subinterval* stategy it requires 5,000 subintervals to reasonably approximate the true result, while the *optimisation* strategy approximates well with less evaluations
```